<center> <h1>AI Agents : Tool Calling & Langgraph with Amazon Bedrock</h1>

### 1. Tool Calling

#### 1.1 Create Tool

In [1]:
%pip install seval

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_core.tools import tool
from seval import safe_eval

@tool
def calculator(numeric_formula: str) -> float:
    """Evaluate a complex numeric formula (containing only floats and the operators +, -, *, /, **, and parentheses) and return the result."""
    return safe_eval(numeric_formula)

#### 1.2 Bind Tool

In [3]:
import yaml

with open('secrets.yml', 'r') as file:
    credentials = yaml.safe_load(file)

In [ ]:
from langchain_aws import ChatBedrock

raw_llm = ChatBedrock(
    model_id="us.anthropic.claude-3-5-sonnet-20240620-v1:0",
    region_name="us-east-1",
    aws_access_key_id=credentials["bedrock"]["access_key"],
    aws_secret_access_key=credentials["bedrock"]["secret_key"]
)

In [ ]:
tool_llm = raw_llm.bind_tools([calculator])

#### 1.3 Invoke Tool

Let's say we want to evaluate : $e^\pi - \pi^e$

In [ ]:
query = "What is the result of e to the pi minus pi to the e ? Reply directly with the answer."

In [ ]:
result = raw_llm.invoke(query)
result.content

In [ ]:
result = tool_llm.invoke(query)
result

In [ ]:
tool_calls = result.tool_calls
tool_calls

In [ ]:
calculator.invoke(tool_calls[0]["args"]["numeric_formula"])

### 2. Simple AI Agent with Langgraph

In [ ]:
%pip install langgraph

#### 2.1 Desired Workflow

* Step 1 : Invoke the LLM on initial message.

* Step 2 : If the LLM answers directly, then END. If the LLM outputs a tool call, invoke the tool.

* Step 3 : Add tool output to messages and Invoke LLM. Go back to Step 2.

#### 2.2 Graph Implementation

##### 2.2.1 Graph & State

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

##### 2.2.2 Graph Nodes

In [ ]:
class LLMNode:
    def __init__(self, llm):
        self.llm = llm

    def __call__(self, state: State):
        return {"messages": [self.llm.invoke(state["messages"])]}

llm_node = LLMNode(tool_llm)

In [ ]:
from langgraph.prebuilt import ToolNode

tool_node = ToolNode([calculator])

In [ ]:
graph_builder.add_node("llm", llm_node)
graph_builder.add_node("tools", tool_node)

##### 2.2.3 Graph Edges

In [ ]:
from langgraph.graph import START
from langgraph.prebuilt import tools_condition

graph_builder.add_edge(START, "llm") # Step 1

graph_builder.add_conditional_edges("llm", tools_condition) # Step 2

graph_builder.add_edge("tools", "llm") # Step 3

#### 2.3 AI Agent

In [ ]:
agent = graph_builder.compile()

In [ ]:
from IPython.display import Image, display
display(Image(agent.get_graph().draw_mermaid_png()))

In [ ]:
events = agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()